In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np
np.random.seed(1337)

import gc
import glob
import cv2
import datetime
import pandas as pd
import time
import scipy
import warnings
warnings.filterwarnings("ignore")
from scipy import misc
import matplotlib.pyplot as plt
%matplotlib inline

from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from sklearn.model_selection import train_test_split

from models import *
from models_utils_loading import *
from models_utils_fit import *
from models_utils_inmem import *
from clr_callback import *

import inception_v4

In [4]:
def fit_model(X, X_tr, X_val, y_tr, y_val, modelname = None, checkname = None, load_name = None):
    callbacks = [ModelCheckpoint('/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/scripts/models/checks/{}.h5'.format(checkname), 
                                        monitor='val_loss', 
                                        verbose = 0, save_best_only = True),
                EarlyStopping(monitor='val_loss', patience = 12, verbose = 1),
                ReduceLROnPlateau(monitor='val_loss', factor = 0.4, verbose = 1, 
                                  patience = 4, min_lr = 5e-6)]
    
    test_datagen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.05,
                zoom_range=0.15,
                rotation_range=180,
                width_shift_range=0.05,
                height_shift_range=0.05,
                horizontal_flip=True,
                vertical_flip=True)
    train_datagen.fit(X, augment = True)
    valid_datagen = ImageDataGenerator(rescale=1./255,)
    
    if 'res' or 'inception' in checkname:
        batch_size = 8
    if 'xception' in checkname:
        batch_size = 4
        
    if load_name is not None:
        model = load_model(checks_src + load_name)
    else:
        model = modelname()
        
    model.fit_generator(train_datagen.flow(X_tr, y_tr, batch_size = batch_size), 
                        steps_per_epoch = X_tr.shape[0]/batch_size,
                        validation_data = valid_datagen.flow(X_val, y_val, batch_size = batch_size, shuffle = False),
                        validation_steps = X_val.shape[0]/batch_size, epochs = 200, callbacks = callbacks)
    
    return model


In [5]:
src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/data/training_data/'
checks_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/scripts/models/checks/'
sub_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/submissions/Raw/'
sub_dst = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Intel_Cervix/submissions/'


In [4]:
X, _, _ = load_train(src + 'train_crops_170epochs/')

X_tr, y_tr, train_ids = load_train(src + 'train_set/')
y_tr = to_categorical(y_tr)

X_val, y_val, train_ids = load_train(src + 'valid_set/')
y_val = to_categorical(y_val)

X_test, test_ids = load_test(src + 'test_crops_150epochs/test_crops_150epochs/')

Read train images
Load folder Type_1 (Index: 0)
Load folder Type_2 (Index: 1)
Load folder Type_3 (Index: 2)
Read train data time: 10.36 seconds
Read train images
Load folder Type_1 (Index: 0)
Load folder Type_2 (Index: 1)
Load folder Type_3 (Index: 2)
Read train data time: 9.43 seconds
Read train images
Load folder Type_1 (Index: 0)
Load folder Type_2 (Index: 1)
Load folder Type_3 (Index: 2)
Read train data time: 0.91 seconds
Read test images
Read train data time: 3.8 seconds


In [6]:
model = fit_model(X, X_tr, X_val, y_tr, y_val, resnet1, 'resnet1adam_patientsplit_0.2_frcnn170e_crops')

Epoch 1/200
173/173 [==============================] - 60s - loss: 1.4468 - acc: 0.4458 - val_loss: 1.7159 - val_acc: 0.4964
Epoch 2/200
173/173 [==============================] - 51s - loss: 1.2502 - acc: 0.4870 - val_loss: 1.2007 - val_acc: 0.4964
Epoch 3/200
173/173 [==============================] - 50s - loss: 1.1543 - acc: 0.5065 - val_loss: 1.1623 - val_acc: 0.3650
Epoch 4/200
173/173 [==============================] - 49s - loss: 1.0565 - acc: 0.5260 - val_loss: 1.5259 - val_acc: 0.2847
Epoch 5/200
173/173 [==============================] - 49s - loss: 1.0355 - acc: 0.5275 - val_loss: 1.3887 - val_acc: 0.4526
Epoch 6/200
173/173 [==============================] - 50s - loss: 1.0027 - acc: 0.5723 - val_loss: 0.9809 - val_acc: 0.5474
Epoch 7/200
173/173 [==============================] - 51s - loss: 0.9987 - acc: 0.5636 - val_loss: 0.8884 - val_acc: 0.5912
Epoch 8/200
173/173 [==============================] - 49s - loss: 0.9137 - acc: 0.6055 - val_loss: 0.9563 - val_acc: 0.5401


In [7]:
model = fit_model(X, X_tr, X_val, y_tr, y_val, xception, 'xception_patientsplit_0.2_frcnn170e_crops')

Epoch 1/200
346/346 [==============================] - 112s - loss: 1.4850 - acc: 0.3960 - val_loss: 0.9999 - val_acc: 0.5620
Epoch 2/200
346/346 [==============================] - 107s - loss: 1.2106 - acc: 0.4465 - val_loss: 0.9754 - val_acc: 0.5182
Epoch 3/200
346/346 [==============================] - 97s - loss: 1.1149 - acc: 0.4928 - val_loss: 1.0493 - val_acc: 0.4161
Epoch 4/200
346/346 [==============================] - 98s - loss: 1.1330 - acc: 0.4848 - val_loss: 1.1304 - val_acc: 0.4088
Epoch 5/200
346/346 [==============================] - 98s - loss: 1.0562 - acc: 0.5325 - val_loss: 1.1010 - val_acc: 0.5109
Epoch 6/200
346/346 [==============================] - 98s - loss: 1.0301 - acc: 0.5462 - val_loss: 1.0851 - val_acc: 0.5255
Epoch 7/200
345/346 [============================>.] - ETA: 0s - loss: 0.9850 - acc: 0.5572
Epoch 00006: reducing learning rate to 3.9999998989515007e-05.
346/346 [==============================] - 99s - loss: 0.9845 - acc: 0.5578 - val_loss: 1.303

In [8]:
model = fit_model(X, X_tr, X_val, y_tr, y_val, xception2, 'xception2_patientsplit_0.2_frcnn170e_crops')

Epoch 1/200
346/346 [==============================] - 86s - loss: 0.8840 - acc: 0.5918 - val_loss: 0.9249 - val_acc: 0.5547
Epoch 2/200
346/346 [==============================] - 69s - loss: 0.8201 - acc: 0.6286 - val_loss: 1.0067 - val_acc: 0.5401
Epoch 3/200
346/346 [==============================] - 70s - loss: 0.7634 - acc: 0.6553 - val_loss: 0.8267 - val_acc: 0.5985.655
Epoch 4/200
346/346 [==============================] - 69s - loss: 0.7465 - acc: 0.6777 - val_loss: 0.9922 - val_acc: 0.5328
Epoch 5/200
346/346 [==============================] - 69s - loss: 0.6845 - acc: 0.7066 - val_loss: 0.9088 - val_acc: 0.5912
Epoch 6/200
346/346 [==============================] - 69s - loss: 0.6843 - acc: 0.6980 - val_loss: 0.9326 - val_acc: 0.5839
Epoch 7/200
346/346 [==============================] - 69s - loss: 0.6683 - acc: 0.7182 - val_loss: 0.8493 - val_acc: 0.6131
Epoch 8/200
345/346 [============================>.] - ETA: 0s - loss: 0.6374 - acc: 0.7362- ETA:
Epoch 00007: reducing l

In [ ]:
model = fit_model(X, X_tr, X_val, y_tr, y_val, inception1, 'inception1_patientsplit_0.3_origdata')

In [ ]:
model = fit_model(X, X_tr, X_val, y_tr, y_val, inception3, 'inception3_patientsplit_0.3_origdata')

In [ ]:
X_test, test_ids = load_test(src + 'test_crops_150epochs/test_crops_150epochs/')
model = load_model(checks_src + 'resnet1adam_patientsplit_0.2_frcnn170e_crops.h5')
bag_preds = predict_data(model, X_test, 1, 15)
submission_inmem(bag_preds, test_ids, 'xception_patientsplit_origdata')
prep_sub('xception_patientsplit_origdata')